In [386]:
import os
import sys
import numpy as np
import pandas as pd
import subprocess
import matplotlib as mpl
mpl.use('Agg')
import matplotlib.pyplot as plt
from matplotlib.legend_handler import HandlerLine2D

import itertools
from susipop import filter
from susipop import reader
from IPython.html.widgets import FloatProgress
from IPython.display import display
from ipywidgets import interactive
import ipywidgets as widgets
%matplotlib notebook

import plotly
import susipop as sp

from multiprocessing import Pool
from scipy.interpolate import griddata
def get_folder_list():
    f = next(os.walk('.'))[1]
    return f

#  rdf frictional particles at a time

In [387]:

def rdf(path_add,phival, shearrate, bins,time):
    path = path_add+"/rhor1_phi"+str(phival)+"_yd"+str(float(shearrate))+"/"
    dataset = sp.reader.DataSet(path,particles=True, fluid=False)
    print "available timesteps =", len(dataset.tlist)

    
#         print t 
#         print path
    dataset.load_state(dataset.tlist[time])
    idx = np.where(np.array([np.linalg.norm(i) for i in dataset.particles.sshear_fric]) != 0)[0]
    positions = dataset.particles.r[idx]
#         fluid_velocity = dataset.fluid.u
    L = dataset.L # Domain size
    LEs = dataset.LEs  # Lees-Edwards shift at time dataset.t
    LEu = dataset.LEu  # Lees-Edwards velocity

    sim_chars = sp.susi.extract_sim_characteristics(dataset)
    sim_chars['histogram'] = {'bins': bins, 'range': (0, 2.8376564979553224e-05)}
    sim_chars['ball_pid'] = sim_chars['ball_pid']
    sim_chars['radius'] = sim_chars['radius'][idx]
    _, distances = sp.particles.create_particle_particle_distances(positions, **sim_chars)
    sim_chars['distances'] = distances

    rdf = sp.spatial.radial_distribution_function(positions,**sim_chars)
    plt.figure(figsize=[5,5])
    plt.plot(rdf[0]/(2*np.min(sim_chars['radius'])),rdf[1],"g",label="rdf")
    plt.bar(rdf[0]/np.min(2*np.min(sim_chars['radius'])),rdf[1],width = 1.0/sim_chars['histogram']['bins'])


    # s = sim_chars['radius'].min()
    # b = sim_chars['radius'].max()

    s = 4
    b = 11.2/2
    import inspect, re

    def plot_vline(loc, text):
        plt.axvline(loc/float(2*s),linewidth=1,color="k")
        plt.text(loc/float(2*s),3,text, ha = 'right',rotation=90,fontsize=14)

    plot_vline(s+s,"s+s")
    plot_vline(s+b,"s+b")
    plot_vline(b+b,"b+b")

    # plot_vline(s+s+s,"s+s+s")
    # plot_vline(s+s+b,"s+s+b")
    # plot_vline(s+b+b,"s+b+b")
    # plot_vline(b+b+b,"b+b+b")

    # plot_vline(s+s+s+s,"s+s+s+s")
    # plot_vline(s+s+s+b,"s+s+s+b")
    # plot_vline(s+s+b+b,"s+s+b+b")
    # plot_vline(s+b+b+b,"s+b+b+b")
    # plot_vline(b+b+b+b,"b+b+b+b")

    plt.legend(prop={'size':16})
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(1.85, 2.7, "s = small particle \nb = big particle \nvertical lines represent the expected \npeaks in RDF due to intersection of \nthe particles"
             , fontsize=14, verticalalignment='top', bbox=props, wrap=True)
    plt.xlabel("2Rmin",fontsize=14)
    plt.ylabel("g(r)")
    plt.xticks(fontsize=10)
#     plt.title(path)
    #plt.savefig("radial_distribution_function")


v = interactive(rdf, \
                 path_add = widgets.Text(value="./phase_separated_flow/F_CL_2e-10_ratio_10.0"),
                phival = widgets.ToggleButtons(options=[40,45,50,54,55,56,58],value=56,disabled=False), \
                shearrate= widgets.ToggleButtons(options=[1,10,11,11.5,12,12.5,13,14,15,16,18,20,30,50,70,100,200,300,1000],value = 12),\
                bins=(10,100),
                time = widgets.IntSlider(max = -1, min = -10000, value = -10),__manual=True)

display(v)

In [388]:
# path = "rhor1_phi56_yd10.0/"
# dataset = sp.reader.DataSet(path,particles=True, fluid=True)
# dataset.load_state(dataset.tlist[-10])
# positions = dataset.particles.r
# idx = np.where(dataset.particles.sshear_fric == 0)[0]
# sim_chars = sp.susi.extract_sim_characteristics(dataset)
# _, distances = sp.particles.create_particle_particle_distances(positions, **sim_chars)
# len(distances), len(idx)

# hist_kwdict = {}
# hist_kwdict.update(sim_chars.get('histogram', dict()))
# if ("range" not in hist_kwdict) and (L is not None):
#     hist_kwdict['range'] = (0, np.max(sim_chars['L']))

# radial_df, bin_edges = np.histogram(distances, **hist_kwdict)
# count2 = []
# loc2 = []
# for i in range(len(count2)):
#     area = 2*3.14*0.5*(loc[i]+loc[i+1])*2*min(sim_chars['radius'])
#     count2.append(count2[i]/area)
#     loc2.append(0.5*(loc[i]+loc[i+1]))
# # plt.figure()

# average rdf frictional particles over time

In [389]:
progress_ = widgets.FloatProgress(min=0,max=100,value=0)
display(progress_)
def rdf(path_add,phival, shearrate, bins,time):
    path = path_add+"/rhor1_phi"+str(phival)+"_yd"+str(float(shearrate))+"/"
    dataset = sp.reader.DataSet(path,particles=True, fluid=False)
    print "available timesteps =", len(dataset.tlist)
    rdf_mean = []
    valid = 0
    for i,t in enumerate(range(time[0], time[1])):
#         try:
        
        progress_.value=float(i)/len(range(time[0], time[1]))*100
#         print t 
#         print path
        dataset.load_state(dataset.tlist[t])
        idx = np.where(np.array([np.linalg.norm(i) for i in dataset.particles.sshear_fric]) != 0)[0]
        if len(idx)>0:
            valid+=1
            positions = dataset.particles.r[idx]
    #         fluid_velocity = dataset.fluid.u
            L = dataset.L # Domain size
            LEs = dataset.LEs  # Lees-Edwards shift at time dataset.t
            LEu = dataset.LEu  # Lees-Edwards velocity

            sim_chars = sp.susi.extract_sim_characteristics(dataset)
            sim_chars['histogram'] = {'bins': bins, 'range': (0, 2.8376564979553224e-05)}
            sim_chars['ball_pid'] = sim_chars['ball_pid']
            sim_chars['radius'] = sim_chars['radius'][idx]
            _, distances = sp.particles.create_particle_particle_distances(positions, **sim_chars)
            sim_chars['distances'] = distances

            rdf = sp.spatial.radial_distribution_function(positions,**sim_chars)
#             print rdf
            if t==time[0]:
    #             print t==time[0]
                rdf_mean = np.asarray(rdf)
            else:
    #             print t==time[0]
                rdf_mean[1] += rdf[1]
        else:
            coninue
#         print np.shape(rdf[0]),np.shape(rdf[1])
#         except:
#             valid-=1
#             continue
    
    rdf_mean = np.asarray(rdf_mean)
#     print rdf_mean
#     print valid
    
    rdf_mean[1] /= float(valid)
#     print rdf[1]-rdf_mean[1]
    plt.figure(figsize=[5,5])
    plt.plot(rdf_mean[0]/(2*np.min(sim_chars['radius'])),rdf_mean[1],"g",label="rdf")
    plt.bar(rdf_mean[0]/np.min(2*np.min(sim_chars['radius'])),rdf_mean[1],width = 1.0/sim_chars['histogram']['bins'])


    # s = sim_chars['radius'].min()
    # b = sim_chars['radius'].max()

    s = 4
    b = 11.2/2
    import inspect, re

    def plot_vline(loc, text):
        plt.axvline(loc/float(2*s),linewidth=1,color="k")
        plt.text(loc/float(2*s),3,text, ha = 'right',rotation=90,fontsize=14)

    plot_vline(s+s,"s+s")
    plot_vline(s+b,"s+b")
    plot_vline(b+b,"b+b")

    # plot_vline(s+s+s,"s+s+s")
    # plot_vline(s+s+b,"s+s+b")
    # plot_vline(s+b+b,"s+b+b")
    # plot_vline(b+b+b,"b+b+b")

    # plot_vline(s+s+s+s,"s+s+s+s")
    # plot_vline(s+s+s+b,"s+s+s+b")
    # plot_vline(s+s+b+b,"s+s+b+b")
    # plot_vline(s+b+b+b,"s+b+b+b")
    # plot_vline(b+b+b+b,"b+b+b+b")

    plt.legend(prop={'size':16})
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(1.85, 2.7, "s = small particle \nb = big particle \nvertical lines represent the expected \npeaks in RDF due to intersection of \nthe particles"
             , fontsize=14, verticalalignment='top', bbox=props, wrap=True)
    plt.xlabel("2Rmin",fontsize=14)
    plt.ylabel("g(r)")
    plt.ylim([0,7])
    plt.xticks(fontsize=10)
#     plt.title(path)
    #plt.savefig("radial_distribution_function")


v = interactive(rdf, \
                 path_add = widgets.Text(value="./phase_separated_flow/F_CL_2e-10_ratio_10.0"),
                phival = widgets.ToggleButtons(options=[40,45,50,54,55,56,58],value=56,disabled=False), \
                shearrate= widgets.ToggleButtons(options=[1,10,11,11.5,12,12.5,13,14,15,16,18,20,30,50,70,100,200,300,1000],value = 50),\
                bins=(10,100),
                time = widgets.IntRangeSlider(max = -1, min = -10000, value = [-10,-1]),__manual=True)

display(v)

# average rdf all particles over time

In [390]:
progress_ = widgets.FloatProgress(min=0,max=100,value=0)
display(progress_)
def rdf(path_add,phival, shearrate, bins,time):
    path = path_add+"/rhor1_phi"+str(phival)+"_yd"+str(float(shearrate))+"/"
    dataset = sp.reader.DataSet(path,particles=True, fluid=False)
    print "available timesteps =", len(dataset.tlist)
    rdf_mean = []
    valid = 0
    for i,t in enumerate(range(time[0], time[1])):
#         try:
        
        progress_.value=float(i)/len(range(time[0], time[1]))*100
#         print t 
#         print path
        dataset.load_state(dataset.tlist[t])
#         if len(idx)>0:
        valid+=1
        positions = dataset.particles.r
#         fluid_velocity = dataset.fluid.u
        L = dataset.L # Domain size
        LEs = dataset.LEs  # Lees-Edwards shift at time dataset.t
        LEu = dataset.LEu  # Lees-Edwards velocity

        sim_chars = sp.susi.extract_sim_characteristics(dataset)
        sim_chars['histogram'] = {'bins': bins, 'range': (0, 2.8376564979553224e-05)}
        sim_chars['ball_pid'] = sim_chars['ball_pid']
        sim_chars['radius'] = sim_chars['radius']
        _, distances = sp.particles.create_particle_particle_distances(positions, **sim_chars)
        sim_chars['distances'] = distances

        rdf = sp.spatial.radial_distribution_function(positions,**sim_chars)
#             print rdf
        if t==time[0]:
#             print t==time[0]
            rdf_mean = np.asarray(rdf)
        else:
#             print t==time[0]
            rdf_mean[1] += rdf[1]
#         else:
#             coninue
#         print np.shape(rdf[0]),np.shape(rdf[1])
#         except:
#             valid-=1
#             continue
    
    rdf_mean = np.asarray(rdf_mean)
#     print rdf_mean
#     print valid
    
    rdf_mean[1] /= float(valid)
#     print rdf[1]-rdf_mean[1]
    plt.figure(figsize=[5,5])
    plt.plot(rdf_mean[0]/(2*np.min(sim_chars['radius'])),rdf_mean[1],"g",label="rdf")
    plt.bar(rdf_mean[0]/np.min(2*np.min(sim_chars['radius'])),rdf_mean[1],width = 1.0/sim_chars['histogram']['bins'])


    # s = sim_chars['radius'].min()
    # b = sim_chars['radius'].max()

    s = 4
    b = 11.2/2
    import inspect, re

    def plot_vline(loc, text):
        plt.axvline(loc/float(2*s),linewidth=1,color="k")
        plt.text(loc/float(2*s),3,text, ha = 'right',rotation=90,fontsize=14)

    plot_vline(s+s,"s+s")
    plot_vline(s+b,"s+b")
    plot_vline(b+b,"b+b")

    # plot_vline(s+s+s,"s+s+s")
    # plot_vline(s+s+b,"s+s+b")
    # plot_vline(s+b+b,"s+b+b")
    # plot_vline(b+b+b,"b+b+b")

    # plot_vline(s+s+s+s,"s+s+s+s")
    # plot_vline(s+s+s+b,"s+s+s+b")
    # plot_vline(s+s+b+b,"s+s+b+b")
    # plot_vline(s+b+b+b,"s+b+b+b")
    # plot_vline(b+b+b+b,"b+b+b+b")

    plt.legend(prop={'size':16})
    props = dict(boxstyle='round', facecolor='white', alpha=0.5)
    plt.text(1.85, 2.7, "s = small particle \nb = big particle \nvertical lines represent the expected \npeaks in RDF due to intersection of \nthe particles"
             , fontsize=14, verticalalignment='top', bbox=props, wrap=True)
    plt.xlabel("2Rmin",fontsize=14)
    plt.ylabel("g(r)")
    plt.ylim([0,7])
    plt.xticks(fontsize=10)
#     plt.title(path)
    #plt.savefig("radial_distribution_function")


v = interactive(rdf, \
                 path_add = widgets.Text(value="./phase_separated_flow/F_CL_2e-10_ratio_10.0"),
                phival = widgets.ToggleButtons(options=[40,45,50,54,55,56,58],value=56,disabled=False), \
                shearrate= widgets.ToggleButtons(options=[1,10,11,11.5,12,12.5,13,14,15,16,18,20,30,50,70,100,200,300,1000],value = 50),\
                bins=(10,100),
                time = widgets.IntRangeSlider(max = -1, min = -10000, value = [-10,-1]),__manual=True)

display(v)

In [391]:
#phase separated flow veolocity differences

In [393]:
# # plt.figure()
# for i,ratio in enumerate(["10.0","100.0","1000.0"]):
#     velup_mean = [[],[],[]]
#     veldown_mean =  [[],[],[]]
#     time_ =  [[],[],[]]
#     path = "./phase_separated_flow/F_CL_2e-10_ratio_"+ratio+"/rhor1_phi56_yd50.0/"
#     dataset = sp.reader.DataSet(path,particles=True, fluid=False)
#     for t in range(100,600):
#         dataset.load_state(dataset.tlist[t])
#         mid = 3.2e-5
#         idxup = np.where(dataset.particles.r[:,2] > mid)[0]
#         idxdown = np.where(dataset.particles.r[:,2] <= mid)[0]
#         velup = np.mean(np.linalg.norm(dataset.particles.newton_velocity[idxup]))
#         veldown = np.mean(np.linalg.norm(dataset.particles.newton_velocity[idxdown]))
#         velup_mean[i].append(velup)
#         veldown_mean[i].append(veldown)
#         time_[i].append(dataset.tlist[t])
#     print ratio, np.mean(veldown_mean[i])/np.mean(velup_mean[i])
# #     plt.title("phi=56,yd=50,fcl ratio="+ratio)
# #     plt.plot(time_[i],veldown_mean[i],label = "frictional regime fcl="+ratio)
# #     plt.plot(time_[i],velup_mean[i],label = "non frictional regime fcl="+ratio)
# #     plt.plot(time_[i][:len(velup_mean)],[veldown_mean[i][j]/velup_mean[i][j] for j in range(len(velup_mean))],label = "non-frictional to frictional regime velocity ratio fcl="+ratio)
# # plt.legend()

In [375]:
# def gen(p):
#         return np.random.choice([0, 1], size=1, p=[1-p, p])[0]
# def disp_cells(m,phival,fraction_in_friction):    
#     plt.figure()
#     p = phival*fraction_in_friction 
#     domain = np.zeros((m,m))
#     for i in range(m):
#         for j in range(m):
#             domain[i,j] = gen(p)
#     plt.imshow(domain,interpolation='nearest')

# v = interactive(disp_cells, \
#     m = widgets.IntText(min = 10, max = 1000, value = 100),
#     phival = widgets.FloatSlider(max = 1, min = 0, value = 0.5),
#     fraction_in_friction=widgets.FloatSlider(max = 1, min = 0, value = 0.5) 
#     ,__manual=False)

# display(v)

In [376]:
# plt.figure()
# import networkx as nx

# x = range(10)
# Xi,Yi=np.meshgrid(x,x)
# Xi,Yi = Xi.flatten(),Yi.flatten()
# pos = dict()
# for i in range(len(Xi)):
#     pos[i] = (Xi[i],Yi[i])
# X=nx.Graph()
# X.add_nodes_from(pos.keys())


# # for i in range(20):
# #     X.add_edge(random.randint(1,100),random.randint(1,100))
# #     X.remove_node(i)
# #     print X.neighbors(i+50)
    
# nx.draw(X, pos, node_size=10)
# plt.show()

In [377]:
# plt.figure()
# import scipy.spatial as spatial
# x = range(10)
# x,y=np.mgrid[0:10, 0:10]
# pts = zip(x.ravel(), y.ravel())
# pos = dict()
# for i,val in enumerate(pts):
#     pos[i] = val

# G=nx.Graph()
# G.add_nodes_from(pos.keys())

# points = np.random.random(100).reshape(2,50)
# combined_x_y_arrays = np.dstack([x.ravel(),x.ravel()])[0]
# points_list = list(points.transpose())


# def do_kdtree(combined_x_y_arrays,points):
#     mytree = spatial.cKDTree(combined_x_y_arrays)
#     dist, indexes = mytree.query(points)
#     return indexes

# do_kdtree(combined_x_y_arrays,points_list)



# spatial autocorrelation functions

In [429]:
from scipy import spatial

# path = path_add+"/rhor1_phi"+str(phival)+"_yd"+str(float(shearrate))+"/"
# dataset = sp.reader.DataSet(path,particles=True, fluid=False)
class spatial_autocorrelations:
    def __init__(self, phi, yd):
        self.path =  "rhor1_phi"+str(phi)+"_yd"+str(float(yd))+"/"
        self.dataset = sp.reader.DataSet(self.path, particles = True, fluid=False)
        
        
    def load_state(self,t):
        # t ranges from 1 - inf --> timesteps in the simulation
        self.dataset.load_state(self.dataset.tlist[t])
        self.points = dataset.particles.r
        self.get_kd_tree()
        
    def get_kd_tree(self):
        self.kdtree =  spatial.KDTree(self.points)
        
    def get_nearest_points_list(self,cutoff=1e-7,num_of_nearest_neighbours=None):
        if num_of_nearest_neighbours == None: 
            return self.kdtree.query_ball_tree(self.kdtree,cutoff)
        else:
            return self.kdtree.query(self.points, k=num_of_nearest_neighbours)[1]
        
def Morans_i(coordinates,data):
    #data could be stresses, velcoities etc.
    #points =  x,y,z coords like [[x,y,z],[x,y,z],...] so
    pass

obj = spatial_autocorrelations(phi=50,yd=20)
obj.load_state(100)
obj.get_nearest_points_list(cutoff=9.7e-6,num_of_nearest_neighbours=3)

array([[  0, 101, 174],
       [  1, 194, 121],
       [  2,  92, 149],
       [  3, 196, 116],
       [  4,  94, 162],
       [  5,  94, 162],
       [  6, 178,  68],
       [  7,  76, 221],
       [  8,  75, 105],
       [  9, 130,  78],
       [ 10, 215, 154],
       [ 11, 126,  90],
       [ 12, 141, 192],
       [ 13, 125,  65],
       [ 14, 143,  31],
       [ 15, 109, 209],
       [ 16, 215,  29],
       [ 17, 107, 128],
       [ 18, 151, 184],
       [ 19,  14, 115],
       [ 20, 163, 126],
       [ 21, 111,  58],
       [ 22,  21, 143],
       [ 23,  86, 219],
       [ 24, 108, 221],
       [ 25, 121, 152],
       [ 26, 144,  66],
       [ 27,  45, 118],
       [ 28, 178,   6],
       [ 29, 157, 216],
       [ 30, 207, 210],
       [ 31,  14, 115],
       [ 32, 157,  62],
       [ 33,  21,  97],
       [ 34, 140,  85],
       [ 35, 175,   9],
       [ 36, 194, 104],
       [ 37,  63,  49],
       [ 38, 196,  97],
       [ 39, 109, 120],
       [ 40,  55,  76],
       [ 41, 209